<a href="https://colab.research.google.com/github/aratimo/sentiment-analysis-user-reviews-electronics/blob/main/text%2Bimage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
df = pd.read_json('/content/drive/MyDrive/2-Cell_Phones_and_Accessories_5.json',lines=True)

In [ ]:
df = df.dropna(subset=['reviewText'])
df = df.dropna(subset=['summary'])
null_counts = df.isnull().sum()
print(null_counts)

In [ ]:
df.dropna(subset=['image'], inplace=True)
null_counts = df.isnull().sum()
print(null_counts)

In [ ]:
# Create a boolean mask for rows with only one image URL
mask = df['image'].apply(lambda x: len(x) == 1)

# Filter out unwanted rows using the mask
df = df.loc[mask]

In [ ]:
print(df.shape[0])
null_counts = df.isnull().sum()
print(null_counts)

In [ ]:
df['image'] = df['image'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [ ]:
pd.set_option('display.max_colwidth', None)
print(df['image'])

In [ ]:
rows_to_delete = [198, 284, 794, 823, 848, 915, 965, 1039, 1053, 1172, 2134, 2249, 2402, 2403, 2597, 2860, 2956, 2979, 3114, 3120, 3314, 3725, 4040, 4636, 4946, 4989, 5045, 5049, 5332, 5360, 5389, 5474, 5834, 5905, 6132, 6145, 6336, 6594, 6764, 6989, 7571, 7590, 7711, 7744, 7847, 8177, 8605, 8678, 9039, 9144, 9241, 9302, 9666, 9732]

df = df.drop(df.index[rows_to_delete])

In [ ]:
print(df.shape[0])

In [ ]:
print(df.shape[0])
null_counts = df.isnull().sum()
print(null_counts)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Combine summary and reviewText columns into a single text column
    text = text['summary'] + ' ' + text['reviewText']
    
    # Clean the text
    text = text.lower()
    text = re.sub(r'\d+', '', text) # remove digits
    text = re.sub(r'[^\w\s]', '', text) # remove punctuations
    text = re.sub(r'\s+', ' ', text) # remove extra whitespaces
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]
    
    return ' '.join(tokens)

df['text'] = df.apply(preprocess_text, axis=1)

In [ ]:
import requests
from io import BytesIO
from PIL import Image
import numpy as np
import requests
from io import BytesIO

def extract_image_features(image_urls):
    # Create an empty array to store the features
    features = np.empty((len(image_urls), 2048))

    # Iterate over each image url in the list
    for i, image_url in enumerate(image_urls):
        # Download the image from URL
        response = requests.get(image_url)
        image = tf.keras.preprocessing.image.load_img(BytesIO(response.content), target_size=[299, 299])

        # Convert the image to a numpy array
        image_array = tf.keras.preprocessing.image.img_to_array(image)

        # Normalize the image
        image_array /= 255.0

        # Add a batch dimension
        image_array = np.expand_dims(image_array, axis=0)

        # Extract features from the image
        image_features = model(image_array)

        # Convert the features tensor to a numpy array
        image_features = np.array(image_features).reshape((-1,))

        # Store the features for this image in the features array
        features[i] = image_features

    # Concatenate the features for all images in the list
    features = np.concatenate(features, axis=0)

    return features

df['image_features'] = df['image'].apply(extract_image_features)

In [ ]:
import requests
import tensorflow as tf
import io
from io import BytesIO
from PIL import Image
import numpy as np

def extract_image_features(image_paths):
    # Create an empty array to store the features
    features = np.empty((len(image_paths), 2048))
    
    # Iterate over each image path in the list
    for i, image_path in enumerate(image_paths):
        if image_path.startswith("http"):
            # Download the image from the URL
            response = requests.get(image_path)
            image = tf.keras.preprocessing.image.img_to_array(
                tf.keras.preprocessing.image.load_img(
                    io.BytesIO(response.content), target_size=[299, 299]
                )
            )
        else:
            # Load the image from path
            image = tf.keras.preprocessing.image.load_img(image_path, target_size=[299, 299])
        
        # Normalize the image
        image /= 255.0
        
        # Add a batch dimension
        image = np.expand_dims(image, axis=0)
        
        # Extract features from the image
        image_features = model(image)
        
        # Convert the features tensor to a numpy array
        image_features = np.array(image_features).reshape((-1,))
        
        # Store the features for this image in the features array
        features[i] = image_features
    
    # Concatenate the features for all images in the list
    features = np.concatenate(features, axis=0)
    
    return features

In [ ]:
model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', pooling='avg')
df['image_features'] = df['image'].apply(extract_image_features)

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# Combine the text and image features into a single feature vector
X_text = np.array(df['text'].tolist())
X_text = np.reshape(X_text, (-1, 1))
X_image = np.array(df['image_features'].tolist())
X = np.concatenate((X_text, X_image), axis=1)

In [ ]:
# Convert the overall column into a binary sentiment label
df['sentiment'] = df['overall'].apply(lambda x: 1 if x > 3 else 0)

# Prepare the target variable
y = np.array(df['sentiment'].tolist())

In [ ]:
ratings = df['overall']

In [ ]:
features = np.concatenate((text_features, image_features), axis=1)